![header](https://i.imgur.com/I4ake6d.jpg)


<h1><center><b>Open and Visualize NetCDF files in Python</b></center></h1>


---

# Table of contents
- [1. Context](#Context)
    - [1.1. Import Modules](#Import-Modules)
- [2. Use Case: Atlantic-Iberian Biscay Irish Ocean Physics Data](#Use-Case-Example)
- [3. Data Exploration](#Data-Exploration)
- [4. Plot Data](#Plot-Data)
    - [4.1. Your First 2D maps](#Your-First-2D-maps)
    - [4.2. Temporal Dimension](#Temporal-dimension)
    - [4.3. Depth Profile](#Depth-Profile)
- [5. Data Extraction](#Data-Extraction)
- [6. Conclusion](#Conclusion) 
---

## Context
[Go back to the "Table of contents"](#Table-of-contents)

In the following Jupyter Notebook, you will find the main steps to open, visualize and manipulate <a href=https://marine.copernicus.eu/>Copernicus Marine data</a>, delivered in standard <a href="https://help.marine.copernicus.eu/en/articles/4427604-what-is-the-format-of-copernicus-marine-products-netcdf"><b>NetCDF</b></a> file format, within <b>Python</b>. <br/>

We'll go through a step by step process as listed in the [Table of Contents](#Table-of-contents) above.

<div style='border: solid 0.5px; padding-left:0.7em;'>
<em><h5><b>⚠️ Prerequisites:</b></h5>
All Copernicus Marine products and data access services are free of charges, however to follow this notebook and explore data, you will need a <a href="https://resources.marine.copernicus.eu/registration-form">Copernicus Marine Account</a>.
</em>
    </div>


### Import Modules

There are several packages available in Python to open and plot a netCDF file. 
Here is a table resuming the main modules used in this notebook: 

| Module name | Description |
| :---: | :--- |
| **[`xarray`](https://docs.xarray.dev/en/stable/getting-started-guide/index.html)** | Module to easily work with labelled multi-dimensional arrays |
| **[`matplotlib`](https://matplotlib.org/stable/users/index)** | Library for creating static, animated, and interactive visualizations |
| **[`cartopy`](https://scitools.org.uk/cartopy/)** | Package designed for geospatial data processing in order to produce maps and other geospatial data analyses |
| **[`copernicusmarine`](https://help.marine.copernicus.eu/en/articles/7970514-copernicus-marine-toolbox-installation)** | Allow users to browse and download easily all the products of the Copernicus Marine Data Store |

An existing and detailed documentation is extremly useful and the use is advised. A lot of good examples using <code>xarray</code> and <code>matplotlib</code> are available and easy to understand. 


In [1]:
# Syntax to install modules 
#!pip install xarray

In [ ]:
from IPython.display import IFrame
%matplotlib inline
import matplotlib.pyplot as plt
import getpass
import xarray as xr
# import panel.widgets as pnw
# import panel as pn
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import copernicusmarine

# To avoid warning messages
import warnings
warnings.filterwarnings('ignore')

/home/jkopri/Programs/miniconda3/envs/ioi-project/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

## Use Case Example
For the example of this notebook, we will explore the [Atlantic-Iberian Biscay Irish-Ocean Physics Analysis and Forecast](https://data.marine.copernicus.eu/product/IBI_ANALYSISFORECAST_PHY_005_001/description) Marine product, available from the [Copernicus Marine Data Store](https://resources.marine.copernicus.eu/products), to study the **monthly variation of ocean salinity and temperature over one year** (here the 2021). 

#### _Get more info on the product_
1. In the product information page ([INFORMATION](https://resources.marine.copernicus.eu/product-detail/IBI_ANALYSISFORECAST_PHY_005_001/INFORMATION) sheet) you have access to various information, a general overview of the product and to the product documentation, both technical and scientific.
2. On the ["Data access"](https://data.marine.copernicus.eu/product/IBI_ANALYSISFORECAST_PHY_005_001/services) sheet you have the list of available services to retrieve the desired data.
    
#### Parameters Selection
For our use case, we will focus on the following parameters: 

| Parameter | Value |
| :---: | :---|
| **ProductID** | IBI_ANALYSISFORECAST_PHY_005_001 |
| **DatasetID of interest** | `cmems_mod_ibi_phy_anfc_0.027deg-3D_P1M-m` |
| **Product type** | numerical model |
| **Spatial Resolution** | 0.028° × 0.028° |
| **Temporal resolution** | monthly-mean |
| **Temporal coverage** | from 2021-01-01 to 2021-12-31 |
| **Vertical coverage** |from -5500m to 0m |
| **Ocean Variables of interest** | `thetao`, `so` |

We can therefore already define the different product's information that will be useful to connect and access the service:

In [3]:
## Product's parameter
datasetID = 'cmems_mod_ibi_phy_anfc_0.027deg-3D_P1M-m'

---
## Data Exploration

The Copernicus Marine Service provides several mechanisms for users to access and obtain its products: _[Copernicus Marine Client API - Open a dataset or read a dataframe remotely](https://help.marine.copernicus.eu/en/articles/8287609-copernicus-marine-client-api-open-a-dataset-or-read-a-dataframe-remotely#h_71eee6dbce)_ \
⚠️ For all of them, your Copernicus Marine credentials are **required**.

For our use case, we will access the data with the [copernicusmarine](https://help.marine.copernicus.eu/en/articles/7949409-copernicus-marine-client-introduction) toolbox.

To start, let's first open the **entire dataset**, that we'll store in the variable <code>DS</code>. \
We will see later how to select parameters (cf. [Plot Data](#Plot-Data)).

In [ ]:
DS = copernicusmarine.open_dataset(dataset_id = datasetID)
DS

INFO - 2026-01-07T16:58:29Z - Selected dataset version: "202411"
INFO - 2026-01-07T16:58:29Z - Selected dataset part: "default"
INFO - 2026-01-07T16:58:29Z - Downloading Copernicus Marine data requires a Copernicus Marine username and password, sign up for free at: https://data.marine.copernicus.eu/register


Copernicus Marine username:

And we obtain the overview of the dataset <code>cmems_mod_ibi_phy_anfc_0.027deg-3D_P1M-m</code>.

💡 <em><b>Copernicus Marine Pro Tip:</b> You can click on buttons to interact with the metadata of the file and confirm that it contains the requested subset of data.</em>

<div style='border: solid 0.5px; padding-left:0.7em; padding-bottom:0.7em;' >
    <em><h5><b><em>📌 Note:</em></b></h5>
    We explore here the data via a remote connection, however, the <b>same functions can be applied to a downloeded netCDF file</b>. In this case, the first step would be to open the file from the its folder directory as follows:\
    <center><code>nc_file = xr.open_dataset('my_directory/filename.nc')</code></center>
</em>
    </div>


#### _Get info about the dataset_
If instead we prefer to focus on only some specific information, here are some examples: 

In [ ]:
# General information about the data
DS.info

In [ ]:
# Get the full list of variables available in the dataset
DS.data_vars

In [ ]:
# Get the list of dimensions
DS.coords

In [ ]:
# Get info on one specific variable (here the temperature)
DS.thetao

In [ ]:
# Get info on specific dimensions 
DS.depth, DS.time

---
## Plot Data
#### Your First 2D maps
Now, that we have seen how to connect to the dataset of interest and explore the main information, let's focus on the period and variables we are interested in. 

We create a subsetted dataset, by indicating the needed parameters (cf. [parameters selection](#Parameters-selection)), that we'll extract from our dataset <code>DS</code>, using the <code>.sel()</code> and <code>.isel()</code> functions of <code>xarray</code> (for [more info](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.sel.html)).

In [ ]:
subset = DS[['thetao','so']].sel(time = slice("2021-01-01", "2021-12-31"))
subset

As first simple plot, we choose to visualize the **temperature at the ocean surface, on January 2021**. 

In [ ]:
target_date = "2021-01"     # January 2021
target_depth = 0            # Sea Surface level
subset_T = subset.thetao.isel(depth=target_depth).sel(time = target_date, method = 'nearest')             # method='nearest' -> Method to use for inexact matches (use the nearest valid index value)
subset_T

In [ ]:
## Plot 
subset_T.plot(size = 8)

## Add title
title = f'Atlantic East Coast {subset_T.long_name} on {str(target_date)} at {str(target_depth)} meters'       # set the title
plt.title(title, fontsize=16);

#### _Customized plot_
In the figure above we chose to plot the whole geographical area with the simple <code>xarray</code>'s function <code>.plot()</code>. \
We can however **zoom** on a specific area and **customize** the plot by adding some details and personalising it, using the `matplotlib` package.

For this example, we'll focus on the **Portuguese coasts**, whose we define the geographical coordinates <code>port_lon</code> and <code>port_lat</code>:

In [ ]:
port_lon = slice(-11.77, -7.22)     #longitude
port_lat = slice(36.74, 42.22)    #latitude

In [ ]:
# Open the sea water temperature in the "new" subset
port_T = DS.thetao.sel(longitude = port_lon, latitude = port_lat)
port_T

And we generate the 2D map over this region:

In [ ]:
## Add coastlines and land feature 
f = plt.figure(figsize=(18, 7))                                                                      # define the size of the plot
ax = plt.axes(projection=ccrs.PlateCarree())                                                         # define the projection                                    
ax.coastlines()                                                                                      # add the coastlines
ax.add_feature(cfeature.LAND, zorder=1, edgecolor='k')                                               # add continent
gl = ax.gridlines(draw_labels=True)                                                                  # add gridlines
gl.left_labels = False                                                                               # remove latitude labels on the right
gl.top_labels = False                                                                                # remove longitude labels on the top

## Generate plot with colobar
port_T.sel(time=target_date).isel(depth=target_depth).plot(cmap='bwr')                                                                      # cmap -> colorbar                                                   

## Add title
plt.title(f'Portugal - {port_T.long_name} on {str(target_date)} at {str(target_depth)} meters' , fontsize=15)   # add title to the plot

## Temporal dimension

### _2D maps_
It is also possible to observe the **evolution of a variable over time**, through 2D graphs.

In [ ]:
# Define the target period of interest (from January to December 2021)
target_t_period=slice("2021-01-01","2021-12-31")

In [ ]:
# Call the new subset 
t_evolution = port_T.sel(time=target_t_period).isel(depth=target_depth)
#plot
t_evolution.plot(col="time", col_wrap=6, size=4)     # each row has a maximum of 6 columns

We obtain as many 2D graphs as the number of time steps: <br/>
for our example, we have 12 graphs, one per month, where we can easily observe the monthly evolution of the sea surface temperature.

### _Process a timeseries_
Now, let's create a timeseries of the temperature `thetao` evolution for a geographical **gridpoint**, to observe its monthly evolution.

In [ ]:
# we define the lon/lat coordinates
gridpoint_lon = -9.541       #longitude
gridpoint_lat = 38.718       #latitude

In [ ]:
f = plt.figure(figsize=(10,10))                                                                           # define plot size

## Display the locations of the glider on a mini map
ax_mini_map = f.add_axes([0.5, 1, 0.3, 0.2], projection=ccrs.PlateCarree())                               # create the minimap and define its projection
gl = ax_mini_map.gridlines(draw_labels=True)                                                              # add the lon/lat gridlines
gl.right_labels = False                                                                                   # remove latitude labels on the right
gl.top_labels = False                                                                                     # remove longitude labels on the top

# Customize mini map
ax_mini_map.add_feature(cfeature.LAND, zorder=0, edgecolor='k')                                           # add land mask 
ax_mini_map.set_extent([-10, -9, 38.4, 39], crs = ccrs.PlateCarree())                                      # define the extent of the mini map [lon_min,lon_max,lat_min,lat_max]
ax_mini_map.scatter(gridpoint_lon, gridpoint_lat, 25,'tab:blue',transform=ccrs.PlateCarree())             # plot location of point


## Plot the 1D plot, at the surface and for the target period (year 2021)
t_thetao = t_evolution.sel(longitude=gridpoint_lon, latitude=gridpoint_lat, method='nearest')
t_thetao.plot(size=7)    # define the parameters to plot
plt.title(f"Portugal - Monthly evolution of {t_evolution.long_name} on 2021", fontsize=15)     # add title

We can also compare two variables between them and observe the evolution:

In [ ]:
# Open the new subset for salinity
port_S = DS.so.sel(longitude = port_lon, latitude = port_lat, time = target_t_period).isel(depth = target_depth)
port_S

In [ ]:
port_S = port_S.sel(longitude = gridpoint_lon, latitude = gridpoint_lat, method='nearest')    # define the parameters to plot

Two methods can be used: 
- plotting two graphics, one next to the other

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(1, 2, 1)
t_thetao.plot(color = 'blue')
plt.title(f"{t_thetao.long_name} monthly evolution")

plt.subplot(1, 2, 2)
port_S.plot(color = 'orange')
plt.title(f"{port_S.long_name} monthly evolution")

- plotting the two variables in a same graphic

In [ ]:
plt.figure(figsize=(10,10))
plt.ylabel("Temprature vs Salinity")
plt.title(f"Comparison between Temperature and Salinity at lat={gridpoint_lat} and lon={gridpoint_lon}", fontsize=15)

plt.plot(t_thetao, label = 'Temperature (°C)', color= 'blue')
plt.plot(port_S, label='Salinity (psu)', color = 'orange')
plt.legend()

### Depth Profile
The same plot can be done to observe the depth evolution of a variable on a gridpoint. 

In [ ]:
f = plt.figure(figsize=(10,10))   

# Define the parameters (variable, gridpoint, depth range, time) 
d_thetao = DS.thetao.sel(longitude=gridpoint_lon, latitude=gridpoint_lat, method='nearest').isel(depth=slice(0, -1)).sel(time=target_date)

## Display the locations of the glider on a mini map
ax_mini_map = f.add_axes([0.5, 1, 0.3, 0.2], projection=ccrs.PlateCarree())                               # create the minimap and define its projection
gl = ax_mini_map.gridlines(draw_labels=True)                                                              # add the lon/lat gridlines
gl.right_labels = False                                                                                   # remove latitude labels on the right
gl.top_labels = False                                                                                     # remove longitude labels on the top
# Customize mini map
ax_mini_map.add_feature(cfeature.LAND, zorder=0, edgecolor='k')                                           # add land mask 
ax_mini_map.set_extent([-10, -9, 38.4, 39], crs = ccrs.PlateCarree())                            # define the extent of the mini map [lon_min,lon_max,lat_min,lat_max]
ax_mini_map.scatter(gridpoint_lon, gridpoint_lat, 25,'tab:green',transform=ccrs.PlateCarree())       # plot location of point

#plot the vertical profiles
d_thetao.plot(y = 'depth', yincrease=False, color='green', size=7)    # define the parameters to plot

#title
plt.title(f"Depth profile of {d_thetao.long_name} at lat={gridpoint_lat} and lon={gridpoint_lon}", fontsize=15)

---
## Data Extraction
Access through copernicusmarine _open_dataset_ function creates a remote connection and you can virtually explore the data, but no file is stored on your local disk. If you break the connection, you risk losing all the results you have obtained.

**Do not panic!** \
In this section we will show you how to save data, in any format you want! We will focus on the netCDF file format.

In the [package documentation](https://docs.xarray.dev/en/latest/api.html), an exhaustive list of data conversion formats is available, so feel free to check it out!

In [ ]:
# data storage in NetCDF format 
nc_file = subset_T.to_netcdf("CMEMS_Portugal_2021-monhtly.nc")

---
## 6. Conclusion

<div class="alert alert-block alert-success">
    <b>CONGRATULATIONS!!</b> 👏<br>
  
--- 
    


We hope you enjoyed this training on how to open and explore Copernicus Marine data, for free, thanks to the European Commission.

This training course is over but we'd love to hear from you about how we could improve it (topics, tools, storytelling, format, speed, etc.).
    
<div style='border: solid 0.5px; padding-left:0.7em; padding-bottom:0.7em;' >
<em><h5><b>📌 Note:</b></h5>
   For more application examples and use cases (and if you want to explore the files further), we offer a wide selection of trainings and scripts in our <a href="https://marine.copernicus.eu/services/user-learning-services/tutorials">E-learning materials</a> section!
</em>
    </div>
    
<br />    
If you have any questions and/or problems, please do not hesitate to contact the Copernicus Marine Support team:
<ul>
    <li>through a <b>chat session</b> available in the bottom right corner of any Copernicus Marine webpage</li>
</ul>

[Go back to the "Table of contents"](#Table-of-contents)
</div>